In [24]:
""" 1. import pacakge"""
import copy
import os
import random

import numpy as np
import torch
import torch.nn as nn
from torch import optim
from torch.autograd import Variable
from tqdm import tqdm

from data_process_YW import load_data

In [25]:
""" 2. set up hyper-parameters"""
Batch_size = 1
numWorkers = 0

In [26]:
""" 3. set up device"""
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)

mps


In [27]:
""" 4. load data"""
trainloader, testloader = load_data(Batch_size, numWorkers)
# training set 1000, test set 500
# shape of single image: torch.Size([1, 3, 32, 32])
# shape of single label: torch.Size([1])

data processing...


In [28]:
""" 5. define model, loss function, optimizer"""
"""5.1 agent model"""
import torch.nn as nn
import torch.nn.functional as F

class LocalNet(nn.Module):
    def __init__(self):
        super(LocalNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(3 * 32 * 32, 120)
        self.fc2 = nn.Linear(3 * 32 * 32, 1)

    def forward(self, x):
        x = x.view(-1, 3 * 32 * 32)
        #print(x.shape)
        x = self.fc2(x)
        return x
# mobile_net = LocalNet()
mobile_net = LocalNet().to(device)


In [29]:
criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = optim.SGD(mobile_net.parameters(), lr=0.001, momentum=0.9)

In [30]:
"""5.1.2 train agent model"""
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        # inputs, labels = data
        inputs, labels = data[0].to(device), data[1].to(device)
        print(inputs.shape)
        print(labels)

        # labels=labels*2-1
        labels = torch.reshape(labels, (Batch_size, 1))
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = mobile_net(inputs)
        # outputs = (outputs+1)/2

        # print(labels)
        # print(outputs)

        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:  # print every 200 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')
PATH = './localnet.pth'
torch.save(mobile_net.state_dict(), PATH)

ValueError: Target size (torch.Size([1, 1])) must be the same as input size (torch.Size([64, 1]))

In [8]:
"""5.1.3 test agent model"""
mobile_net = LocalNet().to(device)
mobile_net.load_state_dict(torch.load(PATH))

correct = 0
total = 0
correct_1=0
correct_0=0
with torch.no_grad():
    for data in testloader:
        #images, labels = data
        images, labels = data[0].to(device),data[1].to(device)
        outputs = mobile_net(images)
        #_, predicted = torch.max(outputs.data, 1)
        predicted = [1 if x > 0 else 0 for x in outputs]
        predicted=torch.tensor(predicted).to(device)
        #print(type(predicted))
        #print(type(labels))
        total += labels.size(0)
        correct_1 += (predicted == labels and labels==1).sum().item()
        correct_0+=(predicted == labels and labels==0).sum().item()
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 500 test images: %d %%' % (
    100 * correct / total))
print('Total sample size: %d' %total)
print('correct 1: %d'%correct_1)
print('correct 0: %d'%correct_0)



Accuracy of the network on the 500 test images: 51 %
Total sample size: 400
correct 1: 101
correct 0: 106


In [ ]:
"""5.2 consultant model"""
c1=1
ce=0

class EdgeNet(nn.Module):
    def __init__(self):
        super(EdgeNet, self).__init__()
        
        #rejector
        self.rj1=  nn.Linear(3 * 32 * 32, 120)
        self.rj2=  nn.Linear(120, 1)
        
        #edge classifier
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # reject
        y = x.view(-1, 3 * 32 * 32)
        y = F.relu(self.rj1(y))
        y = self.sigmoid(self.rj2(y))*2-1
        #print(y)
        #m=fixed_mobile_net(x)
        
        #edge classifier
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))*2-1
        #print(x)
    
        return [y,x]

# edge_net = EdgeNet()
edge_net = EdgeNet().to(device)

In [ ]:
import math
#custom loss function
def surrogate_loss(outputs, labels, mobile_outputs):
    # here labels={-1. +1}, so we have to pre-processing before input labels
    r,e=outputs[0],outputs[1]
    #print(r,e)
    #value, predicted = torch.max(mobile_outputs.data, 1)
    mobil_predicted = [1 if x > 0 else -1 for x in mobile_outputs] #m={-1, +1}
    mobil_predicted=torch.tensor(mobil_predicted).to(device)
    
    #we assume P(m!=Y|X=x)=0.3 in this case
    value =0.28  # assume that the error rate for each input x is constant.
    beta = 2
    #alpha=beta*(0+math.sqrt((1-value)-(1-value)*(1-value)))/(1-value)
    alpha=2*(math.sqrt(value-value*value))/(value)
    #alpha=beta*(ce+math.sqrt(4*c1*(value-ce)-4*value*value+8*ce*value-4*ce*ce)/(2*value)
    #print(alpha)
    if mobil_predicted==labels:
        loss=torch.exp(-r-labels*e)
        #print(loss)

    else:
        loss=torch.exp(alpha*r)+torch.exp(-r-labels*e)
    return loss

#criterion=surrogate_loss
optimizer = optim.SGD(edge_net.parameters(), lr=0.0001, momentum=0.9)

In [ ]:
#training
for epoch in range(10):  # loop over the dataset multiple times [2, 5, 10]

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        #inputs, labels = data
        inputs, labels = data[0].to(device),data[1].to(device)
        labels=labels*2-1
        labels=torch.reshape(labels, (Batch_size, 1)).to(device) #this part is ok 
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = edge_net(inputs)
        #print(outputs[0],outputs[1])
        #something wrong happened here.
        
        
        mobile_outputs=mobile_net(inputs) #this part is ok
        #print(mobile_outputs.item())
        #mobile_outputs.requires_grad_(False)
        loss = surrogate_loss(outputs, labels, mobile_outputs.detach()).to(device)
        #print(loss)
        loss.backward()
        optimizer.step()
        
        
        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
        #break
print('Finished Training')

In [ ]:
correct = 0
total = 0
predict_locally=0
predict_remotely=0
with torch.no_grad(): # what does it mean?
    for data in testloader:
        #images, labels = data
        images, labels = data[0].to(device),data[1].to(device)
        edge_outputs = edge_net(images)
        mobile_outputs = mobile_net(images)
        r, e=edge_outputs[0].to(device),edge_outputs[1].to(device)
        #print(mobile_outputs)
        #print(r)
        #print(e)
        
        #if r*2-1>0: #assume that interval of r is [0,1]
        if r>0:
            # predict locally
            predict_locally+=1
            predicted = [1 if x > 0 else 0 for x in mobile_outputs]
            predicted=torch.tensor(predicted).to(device)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        else:
            predict_remotely+=1
            predicted = [1 if x > 0 else 0 for x in e]
            predicted=torch.tensor(predicted).to(device)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()            

print("Decision made locally:%d" %predict_locally)
print("Decision made on edge:%d" %predict_remotely)
print('Accuracy of the network on the 2000 test images: %d %%' % (
    100 * correct / total))

In [ ]:
#only run the edge:
correct = 0
total = 0
predict_locally=0
predict_remotely=0
correct_1=0
correct_0=0
with torch.no_grad():
    for data in testloader:
        #images, labels = data
        images, labels = data[0].to(device),data[1].to(device)
        edge_outputs = edge_net(images)
        mobile_outputs = mobile_net(images).to(device)
        r, e=edge_outputs[0].to(device),edge_outputs[1].to(device)
        #print(mobile_outputs)
        #print(r)
        #print(e)

        predict_remotely+=1
        #print(e)
        predicted = [1 if x > 0 else 0 for x in e]
        predicted=torch.tensor(predicted).to(device)
        #print(predicted)
        total += labels.size(0)
        correct_1 += (predicted == labels and labels==1).sum().item()
        correct_0+=(predicted == labels and labels==0).sum().item()
        correct += (predicted == labels).sum().item()            

print(predict_locally)
print(predict_remotely)
print('Accuracy of the network on the 2000 test images: %d %%' % (
    100 * correct / total))
print('Total sample size: %d' %total)
print('correct 1: %d'%correct_1)
print('correct 0: %d'%correct_0)

In [ ]:
# mobile and edge work together,
# there is unbalance happen to rejection

In [ ]:
correct = 0
total = 0
predict_locally=0
predict_remotely=0
local_1=0
local_0=0
edge_1=0
edge_0=0

local_correct_1=0
local_correct_0=0
edge_correct_0=0
edge_correct_1=0
local_correct_edgeSample_0=0
local_correct_edgeSample_1=0
edge_correct_mobilesample_0=0
edge_correct_mobilesample_1=0
mobile_equal_edge_locally_0=0
mobile_equal_edge_remotely_0=0

with torch.no_grad():
    for data in testloader:
        #images, labels = data
        images, labels = data[0].to(device),data[1].to(device)
        edge_outputs = edge_net(images)
        mobile_outputs = mobile_net(images)
        r, e=edge_outputs[0].to(device),edge_outputs[1].to(device)
        #print(mobile_outputs)
        #print(r)
        #print(e)
        
        #if r*2-1>0: #assume that interval of r is [0,1]
        if r>0:
            # predict locally
            predict_locally+=1
            predicted = [1 if x > 0 else 0 for x in mobile_outputs]
            predicted=torch.tensor(predicted).to(device)
            edge_predicted= [1 if x > 0 else 0 for x in e]
            edge_predicted=torch.tensor(edge_predicted).to(device)
            total += labels.size(0)
            local_1+= (labels==1).sum().item()
            local_0+=(labels==0).sum().item()
            local_correct_0+=(predicted == labels and labels==0).sum().item()
            edge_correct_mobilesample_0+=(edge_predicted == labels and labels==0).sum().item()
            local_correct_1+=(predicted == labels and labels==1).sum().item()
            edge_correct_mobilesample_1+=(edge_predicted == labels and labels==1).sum().item()
            mobile_equal_edge_locally_0+=(predicted == labels and labels==0 and edge_predicted== labels).sum().item()
            correct += (predicted == labels).sum().item()
        else:
            predict_remotely+=1
            predicted = [1 if x > 0 else 0 for x in e]
            local_predicted=[1 if x > 0 else 0 for x in mobile_outputs]
            local_predicted=torch.tensor(local_predicted).to(device)
            predicted=torch.tensor(predicted).to(device)
            total += labels.size(0)
            edge_0+=(labels==0).sum().item()
            edge_1+=(labels==1).sum().item()
            edge_correct_0+=(predicted == labels and labels==0).sum().item()
            local_correct_edgeSample_0+=(local_predicted == labels and labels==0).sum().item()
            mobile_equal_edge_remotely_0+=(local_predicted == labels and labels==0 and predicted == labels).sum().item()
            
            edge_correct_1+=(predicted == labels and labels==1).sum().item()
            local_correct_edgeSample_1+=(local_predicted == labels and labels==1).sum().item()
            correct += (predicted == labels).sum().item()            

print("Decision made locally:%d" %predict_locally)
print("local true 1: %d, accuracy: %d %%, accuracy for edge classifier on those samples %d %%"%(local_1, 100*local_correct_1/local_1,100*edge_correct_mobilesample_1/local_1))
print("local true 0: %d, accuracy: %d %%, accuracy for edge classifier on those samples %d %%"%(local_0, 100*local_correct_0/local_0,100*edge_correct_mobilesample_0/local_0))
print("edge = mobile =0 locally: %d"%(mobile_equal_edge_locally_0))
print("Decision made on edge:%d" %predict_remotely)
print("edge true 1: %d, accuracy:%d %%, accuracy for mobile classifier on those samples %d %%"%(edge_1, 100*edge_correct_1/edge_1, 100*local_correct_edgeSample_1/edge_1))
print("edge true 0: %d, accuracy:%d %%, accuracy for mobile classifier on those samples %d %%"%(edge_0, 100*edge_correct_0/edge_0,100*local_correct_edgeSample_0/edge_0 ))
print("edge = mobile =0 remotely: %d"%(mobile_equal_edge_remotely_0))
print('Accuracy of the network on the 2000 test images: %d %%' % (
    100 * correct / total))
